# Program to find NA storms between 1985-2014 (past) or 2021-2050 (future) from original downloaded file

In [13]:
import numpy as np
import xarray as xr
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from tc_functions.loc_constraints import hrmip_na_filter
from tc_functions.storm_constraints import hrmip_remove_coldcore_tcpts, remove_td
from tc_functions.calculations import change_lon_range
from tc_functions.plotting import plot_tc_trajectories

#all lists must be the same length
general_model = 'EC-Earth3P-HR' 
track_file =   
model = ['EC-Earth3P-HR(19852014)(1)', 'EC-Earth3P-HR(19852014)(2)', 'EC-Earth3P-HR(20212050)(1)', 
         'EC-Earth3P-HR(20212050)(2)']
model_type = ['EC-Earth3P-HR(past)(1)', 'EC-Earth3P-HR(past)(2)', 'EC-Earth3P-HR(future)(1)', 
              'EC-Earth3P-HR(future)(2)'] #MODEL_NAME(future/past)
todays_date = '2022-01-11'  #YYYY-MM-DD

for i in range(len(track_file)):
    
    #convert HighResMIP file into format that will work with remaining programs. Save to NetCDF file

    DS = xr.open_dataset(track_file[i])  #open track_file and extract relevant values
    first_pt = DS.FIRST_PT.values
    num_pts = DS.NUM_PTS.values
    index = DS.index.values
    warm_core = DS.warm_core_indicator.values
    total_tracks = DS.TRACK_ID.values
    lons = DS.lon.values
    lats = DS.lat.values
    max_w = DS.sfcWind.values
    time = DS.time.values
    DS.close()

    lons1 = np.empty((len(total_tracks), max(num_pts)), dtype = type(lons[0]))        #set data types for each array
    lats1 = np.empty((len(total_tracks), max(num_pts)), dtype = type(lats[0]))        #'S' = np.bytes_ (same as IBTrACS)
    max_w1 = np.empty((len(total_tracks), max(num_pts)), dtype = type(max_w[0]))
    time1 = np.empty((len(total_tracks), max(num_pts)), dtype = '|S19')
    warm_core1 = np.empty((len(total_tracks), max(num_pts)), dtype = type(warm_core[0]))

    lons1.fill(np.nan)                       #fill desired arrays with empty values
    lats1.fill(np.nan)
    max_w1.fill(np.nan)
    time1.fill(b'')
    warm_core1.fill(np.nan)

    for j in range(len(total_tracks)):       #loop through all TC tracks
        initial = first_pt[j]                #initialize beginning and end points
        final = initial + num_pts[j]
        lons1[j,0:num_pts[j]] = lons[initial:final]    #copy values into array
        lats1[j,0:num_pts[j]] = lats[initial:final]        
        max_w1[j,0:num_pts[j]] = max_w[initial:final]
        m = initial                                    #initialize time counter and convert time values while copying
        n = 0
        for k in range(num_pts[j]):
            time1[j,n] = bytes(str(time[m]), 'utf-8')
            m += 1
            n += 1
        warm_core1[j,0:num_pts[j]] = warm_core[initial:final]
    
    new_ds1 = xr.Dataset(                              #create new NetCDF file and save it
        data_vars = dict(
        clon = (["stormID", "time"], lons1),
        clat = (["stormID", "time"], lats1),
        vmax_2D = (["stormID", "time"], max_w1),
        time_byte = (["stormID", "time"], time1),
        warm_core = (["stormID", "time"], warm_core1)
        ),
        attrs = dict(
        description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, "
                      f"times, and TC type for all storms in the file {track_file[i]}. "
                      "The file is now in np.ndarray format.",
        author = "Justin Willson",
        creation_date = todays_date
        ),
    )

    new_ds1.to_netcdf(f"HighResMIP/{general_model}/{model_type[i]}.all.storms.nc")
    
    #remove non-NA TCs, coldcore TCs, and TDs from dataset and change lon range. Save to NetCDF file
    
    print('Remove unwanted TCs/points')
    print(np.shape(lons1))
    
    lons2, lats2, max_w2, time2, warm_core2 = hrmip_na_filter(lons1, lats1, max_w1, time1, warm_core1)    
    lons2 = change_lon_range(lons2)                                                               
    lons2, lats2, max_w2, time2 = hrmip_remove_coldcore_tcpts(lons2, lats2, max_w2, time2, warm_core2)   
    lons2, lats2, max_w2, time2 = remove_td(lons2, lats2, max_w2, time2, 
                                            f"HighResMIP/{general_model}/{model_type[i]}.all.storms.nc") 
    
    new_ds2 = xr.Dataset(                          #create new NetCDF file and save it
        data_vars = dict(
        clon = (["stormID", "time"], lons2),
        clat = (["stormID", "time"], lats2),
        vmax_2D = (["stormID", "time"], max_w2),
        time_byte = (["stormID", "time"], time2),
        ),
        attrs = dict(
        description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, "
                     f"and times for all NA storms (tropical in nature, no TDs) in the file {track_file[i]}.",
        author = "Justin Willson",
        creation_date = todays_date
        ),
    )

    new_ds2.to_netcdf(f"HighResMIP/{general_model}/na_basin/{model_type[i]}.all.NA.storms.nc")
    
    #find NA TCs only within the years 1985-2014 (if past) or 2021-2050 (if future)
    
    if model_type[i].count('past') == 1:
        first_year = 1985
    else:
        first_year = 2021
    
    year_list = []
    for j in range(np.shape(time2)[0]):
        time_str = str(time2[j,0].decode('utf-8'))
        year = int(time_str[0:4])
        year_list.append(year)

    first_tc = year_list.index(int(first_year))  #take index of first instance of first year in list
    last_tc = np.shape(time)[0]                  #file goes until 2014 or 2050 so take the last storm as the end

    print(first_tc, last_tc)
    
    current_file = f"HighResMIP/{general_model}/na_basin/{model_type[i]}.all.NA.storms.nc"
    DS1 = xr.open_dataset(current_file) 
    lons3 = DS1.clon.sel(stormID=slice(first_tc, last_tc)).values       
    lats3 = DS1.clat.sel(stormID=slice(first_tc, last_tc)).values
    max_w3 = DS1.vmax_2D.sel(stormID=slice(first_tc, last_tc)).values
    time3 = DS1.time_byte.sel(stormID=slice(first_tc, last_tc)).values
    DS1.close()
    
    new_ds3 = xr.Dataset(                          #create new NetCDF file and save it
        data_vars = dict(
        clon = (["stormID", "time"], lons3),
        clat = (["stormID", "time"], lats3),
        vmax_2D = (["stormID", "time"], max_w3),
        time_byte = (["stormID", "time"], time3),
        ),
        attrs = dict(
        description = "Simple NetCDF file with updated longitude, latitude, maximum wind speed, "
                     f"and times for all NA storms for the model {model[i]}.",
        author = "Justin Willson",
        creation_date = todays_date
        ),
    )

    new_ds3.to_netcdf(f"HighResMIP/{general_model}/na_basin/{model[i]}.NA.storms.nc")
    
    #make a global plot of TC tracks to confirm accuracy
    
    plt.figure(figsize=(12,7))            #create figure using cartopy 
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    ax.add_feature(cf.BORDERS)
    ax.add_feature(cf.LAND) 
    ax.set_title(f'{model[i]} NA TC Tracks', fontsize=20)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0)
    gl.xlabels_top = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}

    plot_tc_trajectories(lons3, lats3)

    plt.savefig(f"HighResMIP/{general_model}/na_basin/A_{model[i]}_NAstorms.png")
    plt.close()

Remove unwanted TCs/points
(1416, 106)
(133, 106)
(133, 106)
103
(103, 106)
48 46154
Remove unwanted TCs/points
(1318, 94)
(133, 94)
(133, 94)
103
(103, 94)
56 43189
Remove unwanted TCs/points
(713, 96)
(83, 96)
(83, 96)
68
(68, 96)
10 23071
Remove unwanted TCs/points
(773, 103)
(76, 103)
(76, 103)
59
(59, 103)
12 26249
